In [5]:
import pandas as pd
import numpy as np

df = pd.read_csv("retail_sales_dataset_cleaned.csv")

df.head(5)

,Transaction ID,Date,Time,City,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount,Inventory,Import Cost,Promotion,Holiday,Check_Total_Amount,Month,Quarter,Hour,Age Group
0,1,2023-11-24,10:40,TP.HCM,CUST001,Male,34,Beauty,3,50,150,79,32.62,No,No,150,11,4,10,26-35
1,2,2023-02-27,15:07,Hải Phòng,CUST002,Female,26,Clothing,2,500,1000,244,324.70,No,No,1000,2,1,15,26-35
2,3,2023-01-13,19:01,TP.HCM,CUST003,Male,50,Electronics,1,30,30,47,23.44,Yes,No,30,1,1,19,36-50
3,4,2023-05-21,18:47,TP.HCM,CUST004,Male,37,Clothing,1,500,500,219,324.95,No,No,500,5,2,18,36-50
4,5,2023-05-06,18:17,Hà Nội,CUST005,Male,30,Beauty,2,50,100,115,32.72,No,No,100,5,2,18,26-35


In [10]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr, f_oneway
import statsmodels.api as sm
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

# Đọc dữ liệu từ file CSV
def load_and_preprocess_data():
    # Giả lập dữ liệu từ file CSV
    data = pd.read_csv('retail_sales_dataset_cleaned.csv')

    # Kiểm tra giá trị null
    if data.isnull().sum().any():
        data = data.dropna()

    # Chuyển đổi biến định tính thành số
    categorical_cols = ['Gender', 'City', 'Product Category', 'Promotion', 'Holiday']
    data_encoded = pd.get_dummies(data, columns=categorical_cols, drop_first=True)

    # Chuyển đổi cột Date thành datetime
    data_encoded['Date'] = pd.to_datetime(data_encoded['Date'])

    return data_encoded

# Phân tích tương quan cho biến định lượng
def analyze_quantitative_correlation(data):
    quantitative_cols = ['Quantity', 'Price per Unit', 'Age', 'Inventory', 'Import Cost', 'Hour']
    correlations = {}

    for col in quantitative_cols:
        if col in data.columns:
            corr, p_value = pearsonr(data[col], data['Total Amount'])
            correlations[col] = {'correlation': corr, 'p_value': p_value}

    return correlations

# Phân tích tương quan cho biến định tính
def analyze_categorical_correlation(data):
    categorical_cols = ['Gender', 'City', 'Product Category', 'Promotion', 'Holiday']
    anova_results = {}

    for col in categorical_cols:
        if col in data.columns:
            groups = [data['Total Amount'][data[col] == category] for category in data[col].unique()]
            f_stat, p_value = f_oneway(*groups)
            anova_results[col] = {'f_stat': f_stat, 'p_value': p_value}

    return anova_results


def build_linear_regression(data):
    # Chọn các biến độc lập (chỉ bao gồm cột số và cột định tính đã mã hóa)
    independent_vars = [
        'Quantity', 'Price per Unit', 'Age', 'Inventory', 'Import Cost', 'Hour'
    ] + [col for col in data.columns if col.startswith(('Gender_', 'City_', 'Product Category_', 'Promotion_', 'Holiday_'))]

    # Loại bỏ các cột không tồn tại trong dữ liệu
    independent_vars = [col for col in independent_vars if col in data.columns]

    # Kiểm tra kiểu dữ liệu của các cột độc lập
    print("Kiểu dữ liệu của các cột độc lập:")
    print(data[independent_vars].dtypes)

    # Đảm bảo tất cả các cột đều là kiểu số
    for col in independent_vars:
        if data[col].dtype not in ['int64', 'float64', 'uint8']:
            print(f"Cột {col} có kiểu dữ liệu {data[col].dtype}, đang chuyển thành số.")
            data[col] = pd.to_numeric(data[col], errors='coerce')

    # Kiểm tra và xử lý giá trị null
    if data[independent_vars].isnull().sum().any():
        print("Cảnh báo: Có giá trị null trong dữ liệu, đang thay thế bằng 0.")
        data[independent_vars] = data[independent_vars].fillna(0)

    # Chọn dữ liệu đầu vào
    X = data[independent_vars]
    y = data['Total Amount']

    # Kiểm tra dữ liệu đầu vào với np.asarray
    X_array = np.asarray(X)
    print("Kiểu dữ liệu của X_array:", X_array.dtype)

    # Thêm hằng số cho mô hình hồi quy
    X = sm.add_constant(X)

    # Xây dựng mô hình
    model = sm.OLS(y, X).fit()

    return model

# Hàm chính để chạy phân tích
def main():
    # Tải và tiền xử lý dữ liệu
    data = load_and_preprocess_data()
    # Kiểm tra kiểu dữ liệu của DataFrame
    data = load_and_preprocess_data()
    print("Kiểu dữ liệu của các cột:")
    print(data.dtypes)

    # Kiểm tra giá trị null
    print("\nGiá trị null trong dữ liệu:")
    print(data.isnull().sum())
    print()
    # Phân tích tương quan biến định lượng
    print("Tương quan biến định lượng với Total Amount:")
    quant_correlations = analyze_quantitative_correlation(data)
    for var, stats in quant_correlations.items():
        print(f"{var}: Correlation = {stats['correlation']:.4f}, p-value = {stats['p_value']:.4f}")

    # Phân tích tương quan biến định tính
    print("\nPhân tích ANOVA cho biến định tính với Total Amount:")
    cat_correlations = analyze_categorical_correlation(data)
    for var, stats in cat_correlations.items():
        print(f"{var}: F-statistic = {stats['f_stat']:.4f}, p-value = {stats['p_value']:.4f}")

    # Xây dựng và đánh giá mô hình hồi quy
    model = build_linear_regression(data)
    print("\nKết quả mô hình hồi quy tuyến tính:")
    print(model.summary())

    # Trích xuất các biến có ý nghĩa thống kê (p-value < 0.05)
    significant_vars = model.pvalues[model.pvalues < 0.05].index.tolist()
    print("\nCác biến có ảnh hưởng đáng kể đến Total Amount (p-value < 0.05):")
    for var in significant_vars:
        if var != 'const':
            print(f"{var}: Coefficient = {model.params[var]:.4f}, p-value = {model.pvalues[var]:.4f}")

if __name__ == "__main__":
    main()

Kiểu dữ liệu của các cột:
Transaction ID                           int64
Date                            datetime64[ns]
Time                                    object
Customer ID                             object
Age                                      int64
Quantity                                 int64
Price per Unit                           int64
Total Amount                             int64
Inventory                                int64
Import Cost                            float64
Check_Total_Amount                       int64
Month                                    int64
Quarter                                  int64
Hour                                     int64
Age Group                               object
Gender_Male                               bool
City_Hà Nội                               bool
City_Hải Phòng                            bool
City_TP.HCM                               bool
City_Đà Nẵng                              bool
Product Category_Clothing         

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [11]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr, f_oneway
import statsmodels.api as sm
from sklearn.preprocessing import LabelEncoder
import warnings

warnings.filterwarnings('ignore')

# Đọc dữ liệu từ file CSV
def load_and_preprocess_data():
    # Giả lập dữ liệu từ file CSV
    data = pd.read_csv('retail_sales_dataset_cleaned.csv')

    # Kiểm tra giá trị null
    if data.isnull().sum().any():
        data = data.dropna()

    # Chuyển đổi biến định tính thành số
    categorical_cols = ['Gender', 'City', 'Product Category', 'Promotion', 'Holiday']
    data_encoded = pd.get_dummies(data, columns=categorical_cols, drop_first=True)

    # Chuyển đổi cột Date thành datetime
    data_encoded['Date'] = pd.to_datetime(data_encoded['Date'])

    return data_encoded

# Phân tích tương quan cho biến định lượng
def analyze_quantitative_correlation(data):
    quantitative_cols = ['Quantity', 'Price per Unit', 'Age', 'Inventory', 'Import Cost', 'Hour']
    correlations = {}

    for col in quantitative_cols:
        if col in data.columns:
            corr, p_value = pearsonr(data[col], data['Total Amount'])
            correlations[col] = {'correlation': corr, 'p_value': p_value}

    return correlations

# Phân tích tương quan cho biến định tính
def analyze_categorical_correlation(data):
    categorical_cols = ['Gender', 'City', 'Product Category', 'Promotion', 'Holiday']
    anova_results = {}

    for col in categorical_cols:
        if col in data.columns:
            groups = [data['Total Amount'][data[col] == category] for category in data[col].unique()]
            f_stat, p_value = f_oneway(*groups)
            anova_results[col] = {'f_stat': f_stat, 'p_value': p_value}

    return anova_results


def build_linear_regression(data):
    # Chọn các biến độc lập (chỉ bao gồm cột số và cột định tính đã mã hóa)
    independent_vars = [
        'Quantity', 'Price per Unit', 'Age', 'Inventory', 'Import Cost', 'Hour'
    ] + [col for col in data.columns if col.startswith(('Gender_', 'City_', 'Product Category_', 'Promotion_', 'Holiday_'))]

    # Loại bỏ các cột không tồn tại trong dữ liệu
    independent_vars = [col for col in independent_vars if col in data.columns]

    # Thêm biến phụ thuộc để kiểm tra kiểu dữ liệu cùng lúc
    all_vars_to_check = independent_vars + ['Total Amount']

    # Kiểm tra và chuyển đổi kiểu dữ liệu cho TẤT CẢ các cột sẽ dùng trong mô hình
    print("Kiểm tra và chuyển đổi kiểu dữ liệu của các cột cho mô hình:")
    for col in all_vars_to_check:
        if col in data.columns:
            original_dtype = data[col].dtype
            # Cố gắng chuyển đổi sang kiểu số, lỗi sẽ thành NaN
            data[col] = pd.to_numeric(data[col], errors='coerce')
            if original_dtype != data[col].dtype:
                 print(f"Cột '{col}': Kiểu dữ liệu thay đổi từ {original_dtype} sang {data[col].dtype}")
        else:
            print(f"Cột '{col}' không tồn tại trong dữ liệu.")


    # Kiểm tra và xử lý giá trị null sau khi chuyển đổi
    null_counts = data[all_vars_to_check].isnull().sum()
    if null_counts.sum() > 0:
        print("\nCảnh báo: Có giá trị null trong dữ liệu sau chuyển đổi, đang thay thế bằng 0.")
        print(null_counts[null_counts > 0])
        data[all_vars_to_check] = data[all_vars_to_check].fillna(0)

    # Chọn dữ liệu đầu vào đã được xử lý
    X = data[independent_vars]
    y = data['Total Amount']

    # Thêm hằng số cho mô hình hồi quy
    X = sm.add_constant(X, has_constant='add') # Sử dụng has_constant='add' để tránh thêm nếu đã có

    # Xây dựng mô hình
    model = sm.OLS(y, X).fit()

    return model

# Hàm chính để chạy phân tích
def main():
    # Tải và tiền xử lý dữ liệu
    data = load_and_preprocess_data()
    # Kiểm tra kiểu dữ liệu của DataFrame sau tiền xử lý ban đầu
    print("Kiểu dữ liệu của các cột sau tiền xử lý ban đầu:")
    print(data.dtypes)

    # Kiểm tra giá trị null sau tiền xử lý ban đầu
    print("\nGiá trị null trong dữ liệu sau tiền xử lý ban đầu:")
    print(data.isnull().sum())
    print()
    # Phân tích tương quan biến định lượng
    print("Tương quan biến định lượng với Total Amount:")
    quant_correlations = analyze_quantitative_correlation(data)
    for var, stats in quant_correlations.items():
        print(f"{var}: Correlation = {stats['correlation']:.4f}, p-value = {stats['p_value']:.4f}")

    # Phân tích tương quan biến định tính
    print("\nPhân tích ANOVA cho biến định tính với Total Amount:")
    cat_correlations = analyze_categorical_correlation(data)
    for var, stats in cat_correlations.items():
        print(f"{var}: F-statistic = {stats['f_stat']:.4f}, p-value = {stats['p_value']:.4f}")

    # Xây dựng và đánh giá mô hình hồi quy
    model = build_linear_regression(data.copy()) # Truyền một bản sao để tránh sửa đổi DataFrame gốc không mong muốn
    print("\nKết quả mô hình hồi quy tuyến tính:")
    print(model.summary())

    # Trích xuất các biến có ý nghĩa thống kê (p-value < 0.05)
    significant_vars = model.pvalues[model.pvalues < 0.05].index.tolist()
    print("\nCác biến có ảnh hưởng đáng kể đến Total Amount (p-value < 0.05):")
    for var in significant_vars:
        if var != 'const':
            print(f"{var}: Coefficient = {model.params[var]:.4f}, p-value = {model.pvalues[var]:.4f}")

if __name__ == "__main__":
    main()

Kiểu dữ liệu của các cột sau tiền xử lý ban đầu:
Transaction ID                           int64
Date                            datetime64[ns]
Time                                    object
Customer ID                             object
Age                                      int64
Quantity                                 int64
Price per Unit                           int64
Total Amount                             int64
Inventory                                int64
Import Cost                            float64
Check_Total_Amount                       int64
Month                                    int64
Quarter                                  int64
Hour                                     int64
Age Group                               object
Gender_Male                               bool
City_Hà Nội                               bool
City_Hải Phòng                            bool
City_TP.HCM                               bool
City_Đà Nẵng                              bool
Product Cat

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [12]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr, f_oneway
import warnings
warnings.filterwarnings('ignore')

# Hàm tải và tiền xử lý dữ liệu
def load_and_preprocess_data(file_path='retail_sales_dataset_cleaned.csv'):
    # Đọc file CSV
    data = pd.read_csv(file_path)

    # Kiểm tra giá trị null
    if data.isnull().sum().any():
        print("Cảnh báo: Có giá trị null trong dữ liệu, đang thay thế bằng 0.")
        data = data.fillna(0)

    # Đảm bảo các cột định lượng là kiểu số
    quantitative_cols = ['Quantity', 'Price per Unit', 'Age', 'Import Cost', 'Hour', 'Total Amount']
    for col in quantitative_cols:
        if col in data.columns:
            data[col] = pd.to_numeric(data[col], errors='coerce')

    return data

# Phân tích tương quan cho biến định lượng với Inventory
def analyze_quantitative_correlation(data):
    quantitative_cols = ['Quantity', 'Price per Unit', 'Age', 'Import Cost', 'Hour', 'Total Amount']
    correlations = {}

    for col in quantitative_cols:
        if col in data.columns and col != 'Inventory':
            corr, p_value = pearsonr(data[col], data['Inventory'])
            correlations[col] = {'correlation': corr, 'p_value': p_value}

    return correlations

# Phân tích tương quan cho biến định tính với Inventory
def analyze_categorical_correlation(data):
    categorical_cols = ['Gender', 'City', 'Product Category', 'Promotion', 'Holiday']
    anova_results = {}

    for col in categorical_cols:
        if col in data.columns:
            groups = [data['Inventory'][data[col] == category] for category in data[col].unique()]
            f_stat, p_value = f_oneway(*groups)
            anova_results[col] = {'f_stat': f_stat, 'p_value': p_value}

    return anova_results

# Hàm chính
def main():
    # Tải và tiền xử lý dữ liệu
    data = load_and_preprocess_data()

    # Phân tích tương quan biến định lượng
    print("Tương quan biến định lượng với Inventory:")
    quant_correlations = analyze_quantitative_correlation(data)
    for var, stats in quant_correlations.items():
        print(f"{var}: Correlation = {stats['correlation']:.4f}, p-value = {stats['p_value']:.4f}")

    # Phân tích tương quan biến định tính
    print("\nPhân tích ANOVA cho biến định tính với Inventory:")
    cat_correlations = analyze_categorical_correlation(data)
    for var, stats in cat_correlations.items():
        print(f"{var}: F-statistic = {stats['f_stat']:.4f}, p-value = {stats['p_value']:.4f}")

if __name__ == "__main__":
    main()

Cảnh báo: Có giá trị null trong dữ liệu, đang thay thế bằng 0.
Tương quan biến định lượng với Inventory:
Quantity: Correlation = 0.0206, p-value = 0.5159
Price per Unit: Correlation = 0.0018, p-value = 0.9556
Age: Correlation = -0.0032, p-value = 0.9196
Import Cost: Correlation = 0.0036, p-value = 0.9103
Hour: Correlation = -0.0409, p-value = 0.1967
Total Amount: Correlation = 0.0110, p-value = 0.7280

Phân tích ANOVA cho biến định tính với Inventory:
Gender: F-statistic = 0.0593, p-value = 0.8076
City: F-statistic = 0.3903, p-value = 0.8157
Product Category: F-statistic = 858.0034, p-value = 0.0000
Promotion: F-statistic = 1.5563, p-value = 0.2125
Holiday: F-statistic = 0.1571, p-value = 0.6919


In [16]:
# prompt: Kiểm viết code kiểm tra xem có các thành phố tên gì

# Tải dữ liệu để kiểm tra các thành phố
data_for_cities = load_and_preprocess_data()

# Kiểm tra các thành phố duy nhất trong cột 'City'
unique_cities = data_for_cities['City'].unique()

print("Các thành phố có trong dữ liệu:")
for city in unique_cities:
    city


Cảnh báo: Có giá trị null trong dữ liệu, đang thay thế bằng 0.
Các thành phố có trong dữ liệu:


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Transaction ID      1000 non-null   int64  
 1   Date                1000 non-null   object 
 2   Time                1000 non-null   object 
 3   City                1000 non-null   object 
 4   Customer ID         1000 non-null   object 
 5   Gender              1000 non-null   object 
 6   Age                 1000 non-null   int64  
 7   Product Category    1000 non-null   object 
 8   Quantity            1000 non-null   int64  
 9   Price per Unit      1000 non-null   int64  
 10  Total Amount        1000 non-null   int64  
 11  Inventory           1000 non-null   int64  
 12  Import Cost         1000 non-null   float64
 13  Promotion           1000 non-null   object 
 14  Holiday             1000 non-null   object 
 15  Check_Total_Amount  1000 non-null   int64  
 16  Month  

In [18]:
# prompt: xóa cột Age Group trong df

# Kiểm tra xem cột 'Age Group' có tồn tại trong DataFrame không
if 'Age Group' in df.columns:
    # Xóa cột 'Age Group'
    df = df.drop('Age Group', axis=1)
    print("Cột 'Age Group' đã được xóa.")
else:
    print("Cột 'Age Group' không tồn tại trong DataFrame.")

# Hiển thị thông tin DataFrame sau khi xóa để xác nhận
df.info()


Cột 'Age Group' đã được xóa.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Transaction ID      1000 non-null   int64  
 1   Date                1000 non-null   object 
 2   Time                1000 non-null   object 
 3   City                1000 non-null   object 
 4   Customer ID         1000 non-null   object 
 5   Gender              1000 non-null   object 
 6   Age                 1000 non-null   int64  
 7   Product Category    1000 non-null   object 
 8   Quantity            1000 non-null   int64  
 9   Price per Unit      1000 non-null   int64  
 10  Total Amount        1000 non-null   int64  
 11  Inventory           1000 non-null   int64  
 12  Import Cost         1000 non-null   float64
 13  Promotion           1000 non-null   object 
 14  Holiday             1000 non-null   object 
 15  Check_Total_Amount  1000 no